# Model (Neural Network)

## Create response variable

In [46]:
x1_train <- 
  sim_data_smooth %>% filter(dataset == "train") %>%
  select(id, x1) %>%
  distinct() %>% 
  pull(x1)

In [47]:
head(x1_train)

[1] 0.6066050 0.5118087 0.9837245 0.8713568 0.7376236 0.7715580

## Create data frame with PCs and response for neural network

In [48]:
rf_jfpca_df <- 
  train_transformed_jfpca$fpca_res$coef %>%
  data.frame() %>%
  rename_all(.funs = function(x) stringr::str_replace(x, "X", "pc")) %>%
  mutate(x1 = x1_train) %>%
  select(x1, everything())

In [49]:
cat(dim(train_transformed_jfpca$fpca_res$coef), "\n")

75 75 


## Fit neural network

In [64]:
pcs <- names(rf_jfpca_df)[grepl("^pc", names(rf_jfpca_df))]

x_train <- as.matrix(rf_jfpca_df[, pcs])
y_train <- rf_jfpca_df$x1
x_test  <- as.matrix(rf_test_df[, pcs])
y_test  <- rf_test_df$x1

In [65]:
x_train <- scale(x_train)
x_test  <- scale(x_test, center = attr(x_train, "scaled:center"),
                 scale = attr(x_train, "scaled:scale"))


In [66]:
set.seed(36)
nn_model <- keras_model_sequential() %>%
  layer_dense(units = 32, activation = "relu", input_shape = ncol(x_train)) %>%
  layer_dense(units = 1)  # output 

In [67]:
nn_model %>% compile(
  optimizer = "adam",
  loss = "mse",
  metrics = c("mae")
)

In [68]:
history <- nn_model %>% fit(
  x_train, y_train,
  epochs = 150,
  batch_size = 8,
  validation_split = 0.2,
  verbose = 1
)